In [ ]:
## Latihan 1

In [43]:
import os
import sys
import pandas as pd

data_folder = os.path.join(os.path.expanduser("C:/Users/Christine Rasi/Downloads/DAMIN/Association Analysis-20191008/MovieLens 100K Dataset/"), "Movielens 100K Dataset", "ml-100k")
ratings_filename= os.path.join(data_folder, "D:/pd/MovieLens 100K Dataset/ml-100k/u.data")

In [44]:
all_ratings = pd.read_csv(ratings_filename, delimiter="\t",
                         header=None, names=["UserID", "MovieID", "Rating", "Datetime"])

In [45]:
all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'], unit='s')

In [46]:
all_ratings[:5]

,UserID,MovieID,Rating,Datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [ ]:
## Latihan 2

In [47]:
all_ratings ["favorable"] = all_ratings["Rating"]>3

In [48]:
all_ratings[10:15]

,UserID,MovieID,Rating,Datetime,favorable
10,62,257,2,1997-11-12 22:07:14,False
11,286,1014,5,1997-11-17 15:38:45,True
12,200,222,5,1997-10-05 09:05:40,True
13,210,40,3,1998-03-27 21:59:54,False
14,224,29,3,1998-02-21 23:40:57,False


In [49]:
ratings = all_ratings[all_ratings['UserID'].isin(range(200))]

In [50]:
favorable_ratings = ratings[ratings["favorable"]]

In [51]:
favorable_reviews_by_users = dict((k, frozenset(v.values))
                                 for k, v in favorable_ratings.groupby("UserID")["MovieID"])


In [52]:
num_favorable_by_movie = ratings[["MovieID", "favorable"]].groupby("MovieID").sum()

In [53]:
num_favorable_by_movie.sort_values(by="favorable", ascending=False).head()

,favorable
MovieID,
50,100.0
100,89.0
258,83.0
181,79.0
174,74.0


In [ ]:
## Latihan 3

In [66]:
frequent_itemsets = {}

In [67]:
min_support = 50

In [68]:
frequent_itemsets[1] = dict((frozenset((movie_id,)), row["favorable"])
                          for movie_id, row in num_favorable_by_movie.iterrows()
                          if row["favorable"] > min_support)
print("There are {} movies with more than {} favorable reviews".format(len(frequent_itemset[1]),
                                                     min_support))

There are 16 movies with more than 50 favorable reviews


In [69]:
from collections import defaultdict


In [70]:
def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
                
    return dict([(itemset, frequency) for itemset, frequency in counts.items()
                if frequency >= min_support])

for k in range(2,20):
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users,
                                                  frequent_itemsets[k-1],
                                                  min_support)
    if len(cur_frequent_itemsets)==0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemset of leght {}". format(len(cur_frequent_itemsets),k))
        frequent_itemsets[k] = cur_frequent_itemsets

I found 93 frequent itemset of leght 2
I found 295 frequent itemset of leght 3
I found 593 frequent itemset of leght 4
I found 785 frequent itemset of leght 5
I found 677 frequent itemset of leght 6
I found 373 frequent itemset of leght 7
I found 126 frequent itemset of leght 8
I found 24 frequent itemset of leght 9
I found 2 frequent itemset of leght 10
Did not find any frequent itemsets of length 11


In [71]:
del frequent_itemsets[1]        

In [ ]:
## Latihan 4

In [73]:
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
        for itemset in itemset_counts.keys():
            for conclusion in itemset:
                premise = itemset - set((conclusion,))
                candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

There are 15285 candidate rules


In [74]:
print(candidate_rules[:5])

[(frozenset({7}), 1), (frozenset({1}), 7), (frozenset({50}), 1), (frozenset({1}), 50), (frozenset({1}), 56)]


In [75]:
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

In [76]:
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else :
                incorrect_counts[candidate_rule] += 1

In [77]:
rule_confidence = {candidate_rule: correct_counts[candidate_rule]/ 
                   float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                  for candidate_rule in candidate_rules}

In [78]:
from operator import itemgetter

sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)
for index in range(5):
    print("Rule #{0}". format(index +1))
    (premise, conclusion) = sorted_confidence[index][0]
    print("Rule: If a person recommends {0} they will also recommed {1}".format(premise, conclusion))
    print("- Confidence : {0:.3f}\n".format(rule_confidence[(premise, conclusion)]))

Rule #1
Rule: If a person recommends frozenset({98, 181}) they will also recommed 50
- Confidence : 1.000

Rule #2
Rule: If a person recommends frozenset({172, 79}) they will also recommed 174
- Confidence : 1.000

Rule #3
Rule: If a person recommends frozenset({258, 172}) they will also recommed 174
- Confidence : 1.000

Rule #4
Rule: If a person recommends frozenset({1, 181, 7}) they will also recommed 50
- Confidence : 1.000

Rule #5
Rule: If a person recommends frozenset({1, 172, 7}) they will also recommed 174
- Confidence : 1.000



In [79]:
movie_name_filename = os.path.join(data_folder, "u.item")
movie_name_data = pd.read_csv(movie_name_filename, delimiter="|", header=None, encoding = "mac-roman")

In [80]:
movie_name_data.columns =["MovieID", "Title", "Realse Date", "Video Realease", "IMDB", "<UNK>", \
                         "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", \
                         "Documentary", "Drama", "Fantasy", "Film_Noir", "Horor", "Musical", \
                         "Mystery", "Romance", "Sci-Fi", "Thriller" ,"War", "Western"]
movie_name_data.head()

,MovieID,Title,Realse Date,Video Realease,IMDB,<UNK>,Action,Adventure,Animation,Children's,...,Fantasy,Film_Noir,Horor,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [81]:
def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["MovieID"] == movie_id]["Title"]
    title = title_object.values[0]
    return title

In [82]:
for index in range(5):
    print("Rule #[0]".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx)for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommmends {0} they will also recommed {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #[0]
Rule: If a person recommmends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommed Star Wars (1977)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Contact (1997), Empire Strikes Back, The (1980) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommed Star Wars (1977)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000



In [ ]:
## Latihan 5

In [84]:
test_dataset = all_ratings[~all_ratings['UserID'].isin(range(200))]
test_favorable = test_dataset[test_dataset["favorable"]]
test_favorable_by_users = dict((k, frozenset(v.values))
    for k, v in test_favorable.groupby("UserID")["MovieID"])

In [85]:
correct_counts=defaultdict(int)
incorrect_counts=defaultdict(int)
for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise,conclusion=candidate_rule
        if conclusion in reviews:
            correct_counts[candidate_rule]+=1
        else:
            incorrect_counts[candidate_rule]+=1

In [86]:
test_confidence ={candidate_rule:
                 (correct_counts[candidate_rule]/
                 float(correct_counts[candidate_rule]+ incorrect_counts[candidate_rule]))
                 for candidate_rule in rule_confidence}

In [87]:
for index in range(5):
    print("Rule #[0]".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx)for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommmends {0} they will also recommed {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")

Rule #[0]
Rule: If a person recommmends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommed Star Wars (1977)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Contact (1997), Empire Strikes Back, The (1980) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommed Star Wars (1977)
 - Confidence: 1.000

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommed Raiders of the Lost Ark (1981)
 - Confidence: 1.000



In [88]:
for index in range(5):
    print("Rule #[0]".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx)for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommmends {0} they will also recommed {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Train Confidence: {0:.3f}\n".format(test_confidence.get((premise, conclusion), -1)))

Rule #[0]
Rule: If a person recommmends Silence of the Lambs, The (1991), Return of the Jedi (1983) they will also recommed Star Wars (1977)
 - Train Confidence: 1.000
 - Train Confidence: 0.540

Rule #[0]
Rule: If a person recommmends Empire Strikes Back, The (1980), Fugitive, The (1993) they will also recommed Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Train Confidence: 0.369

Rule #[0]
Rule: If a person recommmends Contact (1997), Empire Strikes Back, The (1980) they will also recommed Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - Train Confidence: 0.369

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Return of the Jedi (1983), Twelve Monkeys (1995) they will also recommed Star Wars (1977)
 - Train Confidence: 1.000
 - Train Confidence: 0.540

Rule #[0]
Rule: If a person recommmends Toy Story (1995), Empire Strikes Back, The (1980), Twelve Monkeys (1995) they will also recommed Raiders of the Lost Ark (1981)
 - Train Confidence: 1.000
 - T